## 深度學習系列| 解讀LeNet及PyTorch實現
###  from [CSDN](https://blog.csdn.n

In [86]:
import torch
import torch.nn as nn


class LeModel(nn.Module):
    def __init__(self, num_class=10):
        super(LeModel, self).__init__()
        # CONV2d(in_ch, out_ch, k_size)
        self.conv1 = nn.Conv2d(1, 20, 5)   # 1x28x28 -> 20x24x24
        self.pool1 = nn.MaxPool2d(2)    # 20x24x24 -> 20x12x12
        self.conv2 = nn.Conv2d(20, 40, 5)    # 20x12x12 -> 40x8x8
        self.pool2 = nn.MaxPool2d(2)    # 16x8x8 -> 40x4x4
        #self.conv3 = nn.Conv2d(20, 120, 4)  # LeNet的input是32x32，MNIST为28x28，对此修改卷积核尺寸为4x4
        self.FC1 = nn.Linear(640*1*1, 64)
        self.Classifier = nn.Linear(64, num_class)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        #x = torch.tanh(self.conv3(x))
        x = x.view(-1, 640*1*1)
        x = self.FC1(x)
        x = torch.relu(x)
        x = self.Classifier(x)
        return x

model = LeModel(num_class=10)
print(model)

LeModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (FC1): Linear(in_features=640, out_features=64, bias=True)
  (Classifier): Linear(in_features=64, out_features=10, bias=True)
)


In [87]:
import torchsummary

# torchsummary.summary(model, input_size = (1, 28, 28))

In [88]:
import torch
import torch.optim as optim
#from LeNet import LeModel
import torchvision.transforms as transforms
import torch.utils.data
import torch.nn as nn


from torchvision.datasets import MNIST

# MNIST.resources = [
#     ('file:///C:/Users/henry_esslab/MNIST/raw/train-images.idx3-ubyte','f68b3c2dcbeaaa9fbdd348bbdeb94873'),
#     ('file:///C:/Users/henry_esslab/MNIST/raw/train-labels.idx1-ubyte','d53e105ee54ea40749a09fcbcd1e9432'),
#     ('file:///C:/Users/henry_esslab/MNIST/raw/t10k-images.idx3-ubyte','9fb629c4189551a2d022fa330f9573f3'),
#     ('file:///C:/Users/henry_esslab/MNIST/raw/t10k-labels.idx1-ubyte','ec29112dd5afa0611ce80d1b7f02629c')
   
# ]

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'device is {device}')

    transform = transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))])

    train_data = MNIST(root='./torch_v2', train=True,
                       transform=transform, download=True)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

    test_data = MNIST(root='./torch_v2', train=False,
                      transform=transform, download=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)
    #------------------
    
    i = 0
    for images, labels in train_loader:
        i = i + 1
    print(i)
    i = 0
    for images, labels in test_loader:
        i = i + 1
    print(i)

    net = LeModel()
    net.to(device)

    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)
    loss_function = nn.CrossEntropyLoss()

    net.train()
    # .train 是用來啟動 batch norm & drop out
    for epoch in range(10):
        running_loss = 0.0
        for step, (images, labels) in enumerate(train_loader, start=0):
            optimizer.zero_grad()
            images, labels = images.to(device), labels.to(device)
            output = net(images)
            loss = loss_function(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if step % 500 == 499:
                print('epoch: %d | step: %5d | train_loss: %.5f |' % (epoch+1, step+1, running_loss/500))
                running_loss = 0.0

    print('Finished Training !')

    net.eval()
    # .eval 是用來關閉 batch norm & drop out
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = net(images)
            _, predict = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predict == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (
            100 * correct / total))
    print("correct = ", correct)


if __name__ == "__main__":
    main()

device is cuda
3750
625
epoch: 1 | step:   500 | train_loss: 2.12691 |
epoch: 1 | step:  1000 | train_loss: 1.04497 |
epoch: 1 | step:  1500 | train_loss: 0.45314 |
epoch: 1 | step:  2000 | train_loss: 0.34180 |
epoch: 1 | step:  2500 | train_loss: 0.29405 |
epoch: 1 | step:  3000 | train_loss: 0.25984 |
epoch: 1 | step:  3500 | train_loss: 0.23797 |
epoch: 2 | step:   500 | train_loss: 0.19776 |
epoch: 2 | step:  1000 | train_loss: 0.19280 |
epoch: 2 | step:  1500 | train_loss: 0.16871 |
epoch: 2 | step:  2000 | train_loss: 0.16147 |
epoch: 2 | step:  2500 | train_loss: 0.15796 |
epoch: 2 | step:  3000 | train_loss: 0.14186 |
epoch: 2 | step:  3500 | train_loss: 0.14704 |
epoch: 3 | step:   500 | train_loss: 0.11557 |
epoch: 3 | step:  1000 | train_loss: 0.12571 |
epoch: 3 | step:  1500 | train_loss: 0.12755 |
epoch: 3 | step:  2000 | train_loss: 0.11465 |
epoch: 3 | step:  2500 | train_loss: 0.10722 |
epoch: 3 | step:  3000 | train_loss: 0.11424 |
epoch: 3 | step:  3500 | train_loss:

In [89]:
print(model.state_dict())

OrderedDict([('conv1.weight', tensor([[[[ 1.6563e-01,  6.3334e-02, -4.7992e-02, -2.2683e-02, -6.0129e-02],
          [-4.1120e-02, -9.0243e-02,  8.2592e-02, -1.0827e-01, -1.5185e-01],
          [ 8.5585e-02,  1.7002e-01,  1.5753e-01,  8.5149e-02,  1.9356e-01],
          [-1.3267e-01, -1.0448e-01,  2.8872e-02, -6.4907e-02, -8.6202e-02],
          [ 1.3070e-01,  1.4237e-02,  1.8222e-01, -1.3658e-02, -3.7670e-02]]],


        [[[-1.9609e-03,  9.4502e-02, -6.2343e-02, -3.7039e-02,  2.9607e-02],
          [ 9.3575e-02, -4.4024e-02,  1.8271e-01, -1.4949e-01, -1.0058e-01],
          [ 1.2397e-01, -4.4526e-02, -3.6121e-02,  3.0935e-02,  1.6570e-01],
          [-1.5374e-01, -1.5736e-01, -5.0952e-02,  6.0303e-02, -6.4847e-02],
          [-1.9371e-01,  1.4333e-02,  6.7919e-02, -1.7876e-01,  1.1651e-02]]],


        [[[-1.6043e-01,  8.2260e-02, -1.7355e-01,  3.2294e-02, -1.7909e-01],
          [-1.7120e-01,  3.7142e-02,  2.2420e-02, -1.9025e-01,  1.3233e-01],
          [ 1.0714e-01, -2.8777e-03,  

In [90]:
FILE = './MNIST/model/model_state_dict.pt'
torch.save(model.state_dict(), FILE)

In [91]:
torch.cuda.is_available()

True

In [92]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

### print model weight
#### pytorch weight shape : out_ch, in_ch, height, width

In [93]:
# for param in model.parameters():
#     print((param.data))
or_dict = model.state_dict()
print(float(or_dict['conv1.weight'][0][0][0][0]))
#print(model.state_dict())
# for key, value in or_dict.iteritems() :
#     print(key)
print(or_dict.keys())

0.1656285673379898
odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'FC1.weight', 'FC1.bias', 'Classifier.weight', 'Classifier.bias'])


In [94]:
# for layer_name in or_dict.keys():
#     print(layer_name)
#     print("layer shape = ", or_dict[layer_name].shape)
#     if 'conv1' in layer_name and 'weight' in layer_name:
#         print(layer_name)
#         print("layer shape = ", or_dict[layer_name].shape)
        
#         for out_ch in range (or_dict[layer_name].shape[0]):
#             for in_ch in range (or_dict[layer_name].shape[1]):
#                 #print("%dth kernel, %dth channel" % (out_ch, in_ch), file = f)
#                 for h in range (or_dict[layer_name].shape[2]):
#                     for w in range (or_dict[layer_name].shape[3]):
#                         print("(%.15f), " % (float(or_dict[layer_name][out_ch][in_ch][h][w])),end='')
#                     #print ("\n", file = f)
#                 #print("----------------\n", file = f)
        

conv1.weight
layer shape =  torch.Size([20, 1, 5, 5])
conv1.weight
layer shape =  torch.Size([20, 1, 5, 5])
(0.165628567337990), (0.063333615660667), (-0.047992274165154), (-0.022682860493660), (-0.060128718614578), (-0.041119530797005), (-0.090242676436901), (0.082591757178307), (-0.108271837234497), (-0.151852637529373), (0.085584983229637), (0.170022323727608), (0.157529726624489), (0.085148587822914), (0.193555638194084), (-0.132670596241951), (-0.104478739202023), (0.028872132301331), (-0.064906805753708), (-0.086201690137386), (0.130697891116142), (0.014237478375435), (0.182219997048378), (-0.013657733798027), (-0.037669971585274), (-0.001960903406143), (0.094502106308937), (-0.062343269586563), (-0.037039324641228), (0.029607132077217), (0.093575254082680), (-0.044024273753166), (0.182707920670509), (-0.149492263793945), (-0.100583270192146), (0.123966351151466), (-0.044526189565659), (-0.036120966076851), (0.030935361981392), (0.165695562958717), (-0.153744339942932), (-0.15736

In [97]:
f = open("./para/para.h", 'w')
conv_index = 1
fc_index = 1
print("#include \"../../inference/nn.h\"\n\n",file = f)
for layer_name in or_dict.keys():
    if 'conv' in layer_name and 'weight' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double conv%d_w_raw[] = {" %conv_index, file = f)
        for out_ch in range (or_dict[layer_name].shape[0]):
            for in_ch in range (or_dict[layer_name].shape[1]):
                #print("%dth kernel, %dth channel" % (out_ch, in_ch), file = f)
                for h in range (or_dict[layer_name].shape[2]):
                    for w in range (or_dict[layer_name].shape[3]):
                        print("(%.15f), " % (float(or_dict[layer_name][out_ch][in_ch][h][w])),end='',file = f)
                    #print ("\n", file = f)
                #print("----------------\n", file = f)
        print("};\n\n", file = f)
    elif 'conv' in layer_name and 'bias' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double conv%d_b_raw[] = {" %conv_index, file = f)
        conv_index = conv_index + 1
        for ch in range (or_dict[layer_name].shape[0]):
            print("(%.15f), " % (float(or_dict[layer_name][ch])),end='',file = f) 
        print("};\n\n", file = f)
    elif 'weight' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double fc%d_w_raw[] = {" %fc_index, file = f)
        for out_ch in range (or_dict[layer_name].shape[0]):
            for in_ch in range (or_dict[layer_name].shape[1]):
                 print("(%.15f), " % (float(or_dict[layer_name][out_ch][in_ch])),end='',file = f)
        print("};\n\n", file = f)
    elif 'bias' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double fc%d_b_raw[] = {" %fc_index, file = f)
        fc_index = fc_index + 1
        for out_ch in range (or_dict[layer_name].shape[0]):
            print("(%.15f), " % (float(or_dict[layer_name][out_ch])),end='',file = f)
        print("};\n\n", file = f)
        
f.close()
        

conv1.weight
layer shape =  torch.Size([20, 1, 5, 5])
conv1.bias
layer shape =  torch.Size([20])
conv2.weight
layer shape =  torch.Size([40, 20, 5, 5])
conv2.bias
layer shape =  torch.Size([40])
FC1.weight
layer shape =  torch.Size([64, 640])
FC1.bias
layer shape =  torch.Size([64])
Classifier.weight
layer shape =  torch.Size([10, 64])
Classifier.bias
layer shape =  torch.Size([10])
